In [1]:
import google_sheets_api as gs
import pandas as pd

In [2]:
df = gs.read_table_to_df()

In [3]:
df

,CreationTime,OrderId,TelegramId,Name,Type,Place,Day,From,To,Period,Payed
0,2024-08-02 21:28:35.000000,2024-08-03_3h_09-00_p1_468997622,468997622,Андрей,h,1,2024-08-03 00:00:00.000000,2024-08-03 09:00:00.000000,2024-08-03 12:00:00.000000,3,False
1,2024-08-02 21:32:36.227464,2024-08-03_3h_09-00_p2_468997622,468997622,Андрей,b,2,2024-08-03 00:00:00.000000,2024-08-03 09:00:00.000000,2024-08-03 12:00:00.000000,3,False
2,2024-08-02 21:58:45.699025,2024-08-03_1h_12-00_p1_468997622,468997622,Андрей,h,1,2024-08-03 00:00:00.000000,2024-08-03 12:00:00.000000,2024-08-03 13:00:00.000000,1,False
3,2024-08-02 22:40:06.262206,2024-08-03_3h_12-00_p2_468997622,468997622,Андрей,h,2,2024-08-03 00:00:00.000000,2024-08-03 12:00:00.000000,2024-08-03 15:00:00.000000,3,False


In [6]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import os.path
import pickle

In [7]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive',
         'https://www.googleapis.com/auth/calendar',]

credentials = ServiceAccountCredentials.from_json_keyfile_name('ikigai-db-credentials.json', scope)

In [10]:
service = build('calendar', 'v3', credentials=credentials)

# Feature 1: List all calendars
print("Fetching all calendars:")
calendar_list = service.calendarList().list().execute().get('items', [])
print(calendar_list)
for calendar in calendar_list:
    print(calendar['summary'])

Fetching all calendars:
[]


In [12]:
# Feature 2: Create a new calendar
new_calendar = {
    'summary': 'Ikigai Reservations',
    'timeZone': 'Europe/Prague'
}
created_calendar = service.calendars().insert(body=new_calendar).execute()
print(f"Created calendar: {created_calendar['id']}")

Created calendar: f7566d56975c63b9fce4e8388d92c06efc902d6a22cde645ec5792a40aca01c8@group.calendar.google.com


In [17]:
calendar = service.calendars().get(calendarId=created_calendar['id']).execute()
# calendar.share('openzzggl@gmail.com', perm_type='user', role='writer')
calendar

{'kind': 'calendar#calendar',
 'etag': '"4WdwNCuqYdqj3kHPvxvx0GFbgvM"',
 'id': 'f7566d56975c63b9fce4e8388d92c06efc902d6a22cde645ec5792a40aca01c8@group.calendar.google.com',
 'summary': 'Ikigai Reservations',
 'timeZone': 'Europe/Prague'}

In [13]:
# Feature 3: Insert an event
event = {
    'summary': 'AGvTest',
    'location': '800 Howard St., San Francisco, CA 94103',
    'description': 'A meeting to discuss Python projects.',
    'start': {
        'dateTime': (datetime.utcnow() + timedelta(days=1)).isoformat(),
        'timeZone': 'America/Los_Angeles',
    },
    'end': {
        'dateTime': (datetime.utcnow() + timedelta(days=1, hours=1)).isoformat(),
        'timeZone': 'America/Los_Angeles',
    },
}
created_event = service.events().insert(calendarId=created_calendar['id'], body=event).execute()
print(f"Created event: {created_event['id']}")

Created event: jtal9pan13lsbbdhiq3bddfsv0
